In [1]:
import pandas as pd
import numpy as np

In [2]:
# from CNN, these match the cluster_data.csv latitudes and longitudes one-to-one
feats = np.load('aggregated_feats.npy')

In [3]:
clusters = pd.read_csv('cluster_data.csv')

In [4]:
clusters.head()

,lat,lon,cluster_cons,cluster_phone_cons,cluster_hh_f34,cluster_hh_f35,cluster_phone_na,cluster_hh_f34_na,cluster_hh_f35_na,cluster_count,nightlights
0,-17.095150,35.217213,4746.5570,273.444760,0.8750,42.312500,0.0,0.0,0.5000,16,0.000000
1,-17.092351,35.114643,3741.7559,16.226484,0.1250,11.400000,0.0,0.0,0.8750,16,0.000000
2,-17.016698,35.079629,5159.4243,11.216326,0.1250,7.500000,0.0,0.0,0.8750,16,0.000000
3,-16.977243,35.205706,4650.9090,41.399770,0.1875,18.333333,0.0,0.0,0.8125,16,0.121212
4,-16.956385,35.168967,2806.6390,22.526325,0.2500,7.500000,0.0,0.0,0.7500,16,0.502674


In [48]:
total_country = pd.read_csv('../results/malawi/ridge_phone_density/predictions.csv')

In [49]:
total_country.head()

,centroid_lat,centroid_lon,predicted_phone_density
0,-17.083471,35.291291,1.510037
1,-17.083036,35.142291,1.349357
2,-17.082668,35.231750,1.513969
3,-17.068863,35.084458,0.698982
4,-17.013450,35.078743,0.632675


In [53]:
clusters['cluster_hh_f34'].mean()

0.7891634173393223

In [51]:
total_country.mean()

centroid_lat              -13.529790
centroid_lon               34.276004
predicted_phone_density     0.994067
dtype: float64

In [54]:
def find_nearest(x):
    lat = x.centroid_lat
    lon = x.centroid_lon
    
    return ((clusters['lat'] - lat)**2 + (clusters['lon'] - lon)**2).idxmin()
    
    

In [55]:
total_country['nearest'] = total_country.apply(lambda x: find_nearest(x), axis=1)

In [56]:
total_country

,centroid_lat,centroid_lon,predicted_phone_density,nearest
0,-17.083471,35.291291,1.510037,0
1,-17.083036,35.142291,1.349357,1
2,-17.082668,35.231750,1.513969,0
3,-17.068863,35.084458,0.698982,1
4,-17.013450,35.078743,0.632675,2
...,...,...,...,...
1198,-9.509490,33.326193,1.757166,776
1199,-9.509371,33.257111,0.536651,778
1200,-9.508237,33.160050,0.978699,777
1201,-9.486041,33.073519,1.206587,777


In [57]:
res = pd.merge(total_country, clusters[['lat', 'lon', 'cluster_hh_f34']].reset_index().rename(columns={'index': 'nearest'}), on='nearest')



In [58]:
res

,centroid_lat,centroid_lon,predicted_phone_density,nearest,lat,lon,cluster_hh_f34
0,-17.083471,35.291291,1.510037,0,-17.095150,35.217213,0.8750
1,-17.082668,35.231750,1.513969,0,-17.095150,35.217213,0.8750
2,-17.083036,35.142291,1.349357,1,-17.092351,35.114643,0.1250
3,-17.068863,35.084458,0.698982,1,-17.092351,35.114643,0.1250
4,-17.013450,35.078743,0.632675,2,-17.016698,35.079629,0.1250
...,...,...,...,...,...,...,...
1198,-9.565807,33.165787,0.993352,777,-9.519230,33.139193,0.7500
1199,-9.508237,33.160050,0.978699,777,-9.519230,33.139193,0.7500
1200,-9.486041,33.073519,1.206587,777,-9.519230,33.139193,0.7500
1201,-9.509371,33.257111,0.536651,778,-9.507538,33.259649,0.6250


In [7]:
# This is a bunch of code from the Jean et al Github that is modified to work with Python3 and our data

import numpy as np
import pandas as pd
import random
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
from matplotlib.collections import EllipseCollection
import seaborn as sns
import joblib
from sklearn import metrics

def predict_consumption(
    X, y, k=5, k_inner=5, points=10,
        alpha_low=1, alpha_high=5, margin=0.25):
    """
    Plots predicted consumption
    """
    y_hat, r2 = run_cv(X, y, k, k_inner, points, alpha_low, alpha_high)
    return X, y, y_hat, r2


def run_cv(X, y, k, k_inner, points, alpha_low, alpha_high, randomize=False):
    """
    Runs nested cross-validation to make predictions and compute r-squared.
    """
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s = np.zeros((k,))
    y_hat = np.zeros_like(y)
    kf = KFold(n_splits=k, shuffle=True)
    fold = 0
    for train_idx, test_idx in kf.split(X):
        r2s, y_hat, fold = evaluate_fold(
            X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
            randomize)
    return y_hat, r2s.mean()


def scale_features(X_train, X_test):
    """
    Scales features using StandardScaler.
    """
    X_scaler = StandardScaler(with_mean=True, with_std=False)
    X_train = X_scaler.fit_transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test


def train_and_predict_ridge(alpha, X_train, y_train, X_test):
    """
    Trains ridge model and predicts test set.
    """
    ridge = linear_model.Ridge(alpha)
    ridge.fit(X_train, y_train)
    y_hat = ridge.predict(X_test)
    return y_hat


def predict_inner_test_fold(X, y, y_hat, train_idx, test_idx, alpha):
    """
    Predicts inner test fold.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train, X_test = scale_features(X_train, X_test)
    y_hat[test_idx] = train_and_predict_ridge(alpha, X_train, y_train, X_test)
    return y_hat


def find_best_alpha(X, y, k_inner, alphas):
    """
    Finds the best alpha in an inner CV loop.
    """
    kf = KFold(n_splits=k_inner, shuffle=True)
    best_alpha = 0
    best_r2 = 0
    for idx, alpha in enumerate(alphas):
        y_hat = np.zeros_like(y)
        for train_idx, test_idx in kf.split(X):
            y_hat = predict_inner_test_fold(
                X, y, y_hat, train_idx, test_idx, alpha)
        r2 = stats.pearsonr(y, y_hat)[0] ** 2
        if r2 > best_r2:
            best_alpha = alpha
            best_r2 = r2
    print('best alpha', best_alpha)
    return best_alpha


def evaluate_fold(
    X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
        randomize):
    """
    Evaluates one fold of outer CV.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if randomize:
        random.shuffle(y_train)
    best_alpha = find_best_alpha(X_train, y_train, k_inner, alphas)
    X_train, X_test = scale_features(X_train, X_test)
    y_test_hat = train_and_predict_ridge(best_alpha, X_train, y_train, X_test)
    r2 = stats.pearsonr(y_test, y_test_hat)[0] ** 2
    r2s[fold] = r2
    y_hat[test_idx] = y_test_hat
    return r2s, y_hat, fold + 1


In [8]:
y = clusters['cluster_cons'].values
y_log = np.log(y + 0.0001)

In [9]:
_, _, y_hat_log, r2 = predict_consumption(feats, y_log)
r2

best alpha 27.825594022071243
best alpha 77.4263682681127
best alpha 77.4263682681127
best alpha 10.0
best alpha 77.4263682681127


0.42159020164005884

In [10]:
_, _, y_hat, r2 = predict_consumption(feats, y)
r2

best alpha 599.4842503189409
best alpha 599.4842503189409
best alpha 599.4842503189409
best alpha 1668.100537200059
best alpha 599.4842503189409


0.29743908591587404

In [11]:
clusters['predicted_cons'] = y_hat
clusters['predicted_log_cons'] = y_hat_log

In [12]:
ss = StandardScaler()
scaled_feats = ss.fit_transform(feats)

In [13]:
joblib.dump(ss, 'scaler.joblib')

['scaler.joblib']

In [14]:
chosen_alpha = 600
consumption_ridge = linear_model.Ridge(alpha=chosen_alpha)
consumption_ridge.fit(scaled_feats, y)

preds = consumption_ridge.predict(scaled_feats)
metrics.r2_score(y, preds)

0.21655987457377524

In [15]:
joblib.dump(consumption_ridge, 'ridge_consumption.joblib')

['ridge_consumption.joblib']

In [16]:
y = clusters['cluster_phone_cons'].values
y_log = np.log(y + 0.0001)

In [72]:
_, _, y_hat_log, r2 = predict_consumption(feats, y_log)
r2

best alpha 77.4263682681127
best alpha 599.4842503189409
best alpha 599.4842503189409
best alpha 10.0
best alpha 77.4263682681127


0.15467627308491724

In [73]:
_, _, y_hat, r2 = predict_consumption(feats, y)
r2

best alpha 77.4263682681127
best alpha 77.4263682681127
best alpha 215.44346900318823
best alpha 27.825594022071243
best alpha 77.4263682681127


0.37513281729799025

In [13]:
clusters['predicted_phone_cons'] = y_hat
clusters['predicted_log_phone_cons'] = y_hat_log

In [17]:
chosen_alpha = 77
phone_consumption_ridge = linear_model.Ridge(alpha=chosen_alpha)
phone_consumption_ridge.fit(scaled_feats, y)

preds = phone_consumption_ridge.predict(scaled_feats)
metrics.r2_score(y, preds)

0.5276008343926226

In [18]:
joblib.dump(phone_consumption_ridge, 'ridge_phone_consumption.joblib')

['ridge_phone_consumption.joblib']

In [19]:
y = clusters['cluster_hh_f34'].values
y_log = np.log(y + 0.0001)

In [13]:
_, _, y_hat_log, r2 = predict_consumption(feats, y_log)
r2

best alpha 215.44346900318823
best alpha 27.825594022071243
best alpha 27.825594022071243
best alpha 27.825594022071243
best alpha 77.4263682681127


0.20867570759489765

In [14]:
_, _, y_hat, r2 = predict_consumption(feats, y)
r2

best alpha 10.0
best alpha 27.825594022071243
best alpha 77.4263682681127
best alpha 27.825594022071243
best alpha 27.825594022071243


0.5093559275936034

In [17]:
clusters['predicted_cluster_hh_f34'] = y_hat
clusters['predicted_log_cluster_hh_f34'] = y_hat_log

In [20]:
chosen_alpha = 77
phone_density_ridge = linear_model.Ridge(alpha=chosen_alpha)
phone_density_ridge.fit(scaled_feats, y)

preds = phone_density_ridge.predict(scaled_feats)
metrics.r2_score(y, preds)

0.6654651350951115

In [21]:
joblib.dump(phone_density_ridge, 'ridge_phone_density.joblib')

['ridge_phone_density.joblib']

In [15]:
not_nas = ~clusters['cluster_hh_f35'].isna()

In [16]:
y = clusters['cluster_hh_f35'].values[not_nas]
y_log = np.log(y + 0.0001)

In [17]:
_, _, y_hat_log, r2 = predict_consumption(feats[not_nas], y_log)
r2

best alpha 599.4842503189409
best alpha 215.44346900318823
best alpha 215.44346900318823
best alpha 599.4842503189409
best alpha 215.44346900318823


0.24400354816527306

In [18]:
_, _, y_hat, r2 = predict_consumption(feats[not_nas], y)
r2

best alpha 77.4263682681127
best alpha 215.44346900318823
best alpha 215.44346900318823
best alpha 215.44346900318823
best alpha 599.4842503189409


0.22691466124246557

In [22]:
clusters['predicted_cluster_hh_f35'] = np.nan
clusters['predicted_log_cluster_hh_f35'] = np.nan

clusters['predicted_cluster_hh_f35'].loc[not_nas] = y_hat
clusters['predicted_log_cluster_hh_f35'].loc[not_nas] = y_hat_log

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
clusters.head()

,lat,lon,cluster_cons,cluster_phone_cons,cluster_hh_f34,cluster_hh_f35,cluster_phone_na,cluster_hh_f34_na,cluster_hh_f35_na,cluster_count,nightlights,predicted_cons,predicted_log_cons,predicted_phone_cons,predicted_log_phone_cons,predicted_cluster_hh_f34,predicted_log_cluster_hh_f34,predicted_cluster_hh_f35,predicted_log_cluster_hh_f35
0,-17.095150,35.217213,4746.5570,273.444760,0.8750,42.312500,0.0,0.0,0.5000,16,0.000000,5224.726467,8.364173,197.011657,4.670097,0.641980,-0.879296,33.253433,3.236180
1,-17.092351,35.114643,3741.7559,16.226484,0.1250,11.400000,0.0,0.0,0.8750,16,0.000000,5253.604672,8.346371,77.659524,4.100761,0.445353,-1.013334,31.185990,3.114704
2,-17.016698,35.079629,5159.4243,11.216326,0.1250,7.500000,0.0,0.0,0.8750,16,0.000000,4885.964223,8.295663,76.161017,4.388805,0.369442,-1.074152,27.663547,2.994574
3,-16.977243,35.205706,4650.9090,41.399770,0.1875,18.333333,0.0,0.0,0.8125,16,0.121212,7068.678551,8.611166,220.232924,4.533100,0.668825,-0.673597,34.025161,3.221873
4,-16.956385,35.168967,2806.6390,22.526325,0.2500,7.500000,0.0,0.0,0.7500,16,0.502674,6762.347993,8.331267,141.260136,4.161146,0.509615,-1.200498,33.721625,3.072632


In [28]:
clusters.drop(['cluster_phone_na', 'cluster_hh_f34_na', 'cluster_hh_f35_na'], axis=1).to_csv('cluster_cnn_predictions.csv', index=False)
